In [ ]:
#Notebook Description: This notebooks is used to Forecast individual assets based given a combination of features
#I have found useful across my different studies

In [ ]:
#TODO

# 1.Implement feature selection by using models like Random Forest or XGBoost to filter out 
# features that are not important, but keep features that are highly interactive with each other
# and have a non linear depenedence with the target variable

# 2. Integrate Macroeconomic data into the model based on you domain knowledge of analyzing the markets

# 3. Integrate Fundamental Company data into the model based on your domain knowledge of analyzing the markets

# 4. Find interactive features acrooss fundamental,microeconomic and technical data that can be used to forecast the target variable    

# 2. use a Shap value to understand the importance of each feature in the model as modeled by random forest or XGBoost

# 2. Create non linear features such as differences, products, ratios, powers, polynomial features, etc

# 3. Reduce the number of features by using PCA or other dimensionality reduction techniques


In [19]:
#Load all libaries and set parameters
import yfinance as yf
import pandas as pd
import numpy as np
import warnings

from darts.models import ExponentialSmoothing, AutoARIMA, BlockRNNModel, Prophet
from darts.utils.utils import ModelMode, SeasonalityMode
from darts.metrics import mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries

from statsmodels.tsa.stattools import acf, pacf, coint
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt

from IPython.display import clear_output, display
from sklearn.preprocessing import StandardScaler
from collections import Counter
from arch import arch_model

# Quantapp imports
from Quantapp.Model import Model
from Quantapp.Computation import Computation, SequenceGenerator
from Quantapp.Plotter import Plotter

qm = Model()
qc = Computation()
qp = Plotter()
sequence_generator = SequenceGenerator()


from IPython.display import clear_output, display

# Suppress all warnings
warnings.filterwarnings('ignore')

# Define parameters for the notebook
ticker = 'SPY'
period = '10y'
use_returns = False
time_frame_returns = 1
train_percentage = 0.85
forecast_horizon = 21 # Number of days to forecast into the future
input_length =200 # Number of days to use for training
output_length = forecast_horizon # Number of days to forecast into the future


In [20]:
#Custom Functions

def plot_forecast(target, pred):
    # Create traces
    actual_trace = go.Scatter(
        x=target.time_index,
        y=target.values().flatten(),
        mode='lines',
        name='Actual'
    )
    
    forecast_trace = go.Scatter(
        x=pred.time_index,
        y=pred.values().flatten(),
        mode='lines',
        name='Forecast'
    )
    
    # Calculate mean and standard deviation of target
    mean_value = target.values().flatten().mean()
    std_value = target.values().flatten().std()
    
    mean_trace = go.Scatter(
        x=target.time_index,
        y=[mean_value] * len(target.time_index),
        mode='lines',
        name='Mean',
        line=dict(dash='dash')
    )
    
    # Create standard deviation traces
    std_traces = []
    for i in range(1, 4):
        upper_trace = go.Scatter(
            x=target.time_index,
            y=[mean_value + i * std_value] * len(target.time_index),
            mode='lines',
            name=f'+{i} Std Dev',
            line=dict(dash='dot')
        )
        lower_trace = go.Scatter(
            x=target.time_index,
            y=[mean_value - i * std_value] * len(target.time_index),
            mode='lines',
            name=f'-{i} Std Dev',
            line=dict(dash='dot')
        )
        std_traces.extend([upper_trace, lower_trace])
    
    # Create the figure
    fig = go.Figure()
    
    # Add traces to the figure
    fig.add_trace(actual_trace)
    fig.add_trace(forecast_trace)
    fig.add_trace(mean_trace)
    for trace in std_traces:
        fig.add_trace(trace)
    
    # Update layout
    fig.update_layout(
        title='Linear Regression Forecast',
        xaxis_title='Time',
        yaxis_title='Close',
        legend=dict(x=0, y=1),
        height=800,
    )
    
    # Show the figure
    fig.show()
    
    #create a funciton the plots the backtest results against the actuals
def plot_backtest_results_plotly(actual, predictions):
    """
    Plots actual TimeSeries against predicted TimeSeries using Plotly with subplots,
    including mean and ±1, ±2, ±3 standard deviation lines of the actuals,
    and a residuals plot with mean and ±1, ±2, ±3 standard deviation lines in a separate subplot.
    
    Args:
        actual (TimeSeries): The actual target TimeSeries from the test set.
        predictions (TimeSeries): The predicted TimeSeries from the model.
    
    Returns:
        None: Displays the interactive Plotly plot with subplots.
    """
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np
    
    # Convert TimeSeries to DataFrame
    actual_df = actual.pd_dataframe()
    predictions_df = predictions.pd_dataframe()
    
    # Ensure both actual and predictions have the same columns
    common_columns = actual_df.columns.intersection(predictions_df.columns)
    if not common_columns.any():
        raise ValueError("No common columns found between actual and predictions TimeSeries.")
    
    # Initialize subplots: 2 rows, 1 column
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=("Backtest Predictions vs Actuals with Mean and Standard Deviations",
                        "Residuals (Actual - Predicted) with Mean and Standard Deviations"),
        shared_xaxes=True,
        vertical_spacing=0.15
    )
    
    # Plot actual and predicted values with mean and std deviations in the first subplot
    for column in common_columns:
        # Plot actual values
        fig.add_trace(
            go.Scatter(
                x=actual_df.index,
                y=actual_df[column],
                mode='lines',
                name=f'Actual {column}',
                line=dict(color='blue')
            ),
            row=1, col=1
        )
        
        # Plot predicted values
        fig.add_trace(
            go.Scatter(
                x=predictions_df.index,
                y=predictions_df[column],
                mode='lines',
                name=f'Predicted {column}',
                line=dict(color='red', dash='dash')
            ),
            row=1, col=1
        )
        
        # Calculate mean and standard deviation
        mean = actual_df[column].mean()
        std = actual_df[column].std()
        
        # Plot mean line
        fig.add_trace(
            go.Scatter(
                x=actual_df.index,
                y=[mean] * len(actual_df),
                mode='lines',
                name=f'{column} Mean',
                line=dict(color='green', dash='dash')
            ),
            row=1, col=1
        )
        
        # Plot ±1, ±2, ±3 standard deviation lines
        for i in range(1, 4):
            # Mean +i Std Dev
            fig.add_trace(
                go.Scatter(
                    x=actual_df.index,
                    y=[mean + i * std] * len(actual_df),
                    mode='lines',
                    name=f'{column} Mean +{i} Std Dev',
                    line=dict(color='orange', dash='dot')
                ),
                row=1, col=1
            )
            
            # Mean -i Std Dev
            fig.add_trace(
                go.Scatter(
                    x=actual_df.index,
                    y=[mean - i * std] * len(actual_df),
                    mode='lines',
                    name=f'{column} Mean -{i} Std Dev',
                    line=dict(color='orange', dash='dot')
                ),
                row=1, col=1
            )
    
    # Calculate residuals
    residuals_df = actual_df[common_columns] - predictions_df[common_columns]
    
    # Plot residuals with mean and std deviations in the second subplot
    for column in common_columns:
        # Calculate mean and standard deviation of residuals
        residual_mean = residuals_df[column].mean()
        residual_std = residuals_df[column].std()
        
        # Plot residuals
        fig.add_trace(
            go.Scatter(
                x=residuals_df.index,
                y=residuals_df[column],
                mode='lines',
                name=f'Residuals {column}',
                line=dict(color='purple')
            ),
            row=2, col=1
        )
        
        # Plot mean residual line
        fig.add_trace(
            go.Scatter(
                x=residuals_df.index,
                y=[residual_mean] * len(residuals_df),
                mode='lines',
                name=f'{column} Residual Mean',
                line=dict(color='green', dash='dash')
            ),
            row=2, col=1
        )
        
        # Plot ±1, ±2, ±3 standard deviation lines for residuals
        for i in range(1, 4):
            # Residual Mean +i Std Dev
            fig.add_trace(
                go.Scatter(
                    x=residuals_df.index,
                    y=[residual_mean + i * residual_std] * len(residuals_df),
                    mode='lines',
                    name=f'{column} Residual Mean +{i} Std Dev',
                    line=dict(color='orange', dash='dot')
                ),
                row=2, col=1
            )
            
            # Residual Mean -i Std Dev
            fig.add_trace(
                go.Scatter(
                    x=residuals_df.index,
                    y=[residual_mean - i * residual_std] * len(residuals_df),
                    mode='lines',
                    name=f'{column} Residual Mean -{i} Std Dev',
                    line=dict(color='orange', dash='dot')
                ),
                row=2, col=1
            )
        
        # Add a horizontal line at y=0 for reference
        fig.add_trace(
            go.Scatter(
                x=residuals_df.index,
                y=[0] * len(residuals_df),
                mode='lines',
                name=f'Zero Line {column}',
                line=dict(color='black', dash='dash')
            ),
            row=2, col=1
        )
    
    # Update layout for better visualization
    fig.update_layout(
        title="Backtest Results with Residuals",
        template="plotly_dark",
        hovermode="x unified",
        legend=dict(x=0, y=1),
        height=1400  # Increased height to accommodate additional lines
    )
    
    # Update axes titles
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Returns", row=1, col=1)
    fig.update_yaxes(title_text="Residuals", row=2, col=1)
    
    # Show the plot
    fig.show()

#create a function that takes in stock prices, forecasted returns. and plots the future stock prices given the forecasted returns

    

In [21]:
#Compute Optimal Sequences
import numpy as np
import pandas as pd
import plotly.graph_objects as go

sequences = [21,50,200]

In [22]:
#Step 1: Load raw data and engineer features

ticker_list = [ticker]

data = qc.load_and_prepare_data(
    tickers=ticker_list,
    period=period,
    gen_returns=False,
    gen_log_returns=False,
    gen_cumulative_returns=False,
    train_percentage=train_percentage
)

#reformat indexes to datetime without the minutes and seconds, just the month, day, year
#make sure the index is a DateTimeIndex
for ticker in data.keys():
    data[ticker]['full'].index = pd.to_datetime(data[ticker]['full'].index.date)
    data[ticker]['train'].index = pd.to_datetime(data[ticker]['train'].index.date)
    data[ticker]['test'].index = pd.to_datetime(data[ticker]['test'].index.date)
    
def compute_feature_set(df, sequences, lags=range(1, 21)):
    #df has columns 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'
    #drop columns 'Volume', 'Adj Close'
    df_copy = df.copy().drop(columns=['Volume','Capital Gains','Stock Splits','Dividends'])
    
    features = {}
    windows = sequences #use sequences as windows for computing moving averages
    operations = ['differences', 'products', 'sums', 'ratios'] #operations to perform on pairwise features
    transformations = ['polynomial','exponential','log','root']
    #compute moving averages
    # assign calculation to variable
    
    #standard caluclations
    
    moving_averages = qc.compute_moving_averages(df_copy['Close'], windows=windows, ma_type='simple')
    standard_deviation = qc.compute_volatility(df_copy, windows=windows, method='close-to-close')
    skewness = qc.compute_moments_ex_mean(df_copy['Close'], windows=sequences, moment='skew')
    kurtosis = qc.compute_moments_ex_mean(df_copy['Close'], windows=sequences, moment='kurt')
    sharpe = qc.calculate_risk_adjusted_returns(df_copy['Close'],windows=windows, ratio_type='sharpe')  
    drawdown = qc.compute_drawdowns(df_copy['Close'], windows=windows)   


    features['moving_averages'] = moving_averages
    features['standard_deviation'] = standard_deviation
    features['skewness'] = skewness
    features['kurtosis'] = kurtosis
    features['sharpe'] = sharpe
    features['drawdown'] = drawdown
    
    #print the keys of the features dictionary sequentially

    
    features['lags'] = qc.compute_lags(df_copy, lags=lags)
    
    '''    
    display(features['lags'])
    #sort dictionary by keys
    features = dict(sorted(features.items()))
    for key in features.keys():
        print(key)
    '''

    return features


def compute_features(dataframes, sequences, lags=None):
    if lags is None:
        lags = range(1, len(list(dataframes.values())[0]) + 1)
    
    all_features_dict = {}
    
    for name, df in dataframes.items():
        features = compute_feature_set(df, sequences, lags)
        # Prepend the name of the ticker to all columns in features
        features = {f"{name}_{key}": value for key, value in features.items()}
        #each feature holds a dataframe. prepend the name of the ticker to all columns in the dataframe
        features = {key: value.add_prefix(f"{name}_") for key, value in features.items()}
        all_features_dict.update(features)

    reference_index = list(dataframes.values())[0].index
    aligned_features = qc.align_features_to_index(all_features_dict, reference_index)

    all_features = pd.concat(aligned_features.values(), axis=1)

    return aligned_features, all_features

dataframes = {}
# Retrieve all tickers in data and append them to 'dataframes'
for ticker in data.keys():
    dataframes[ticker] = data[ticker]['full']

tickers = list(data.keys())



features_subsets, all_features = compute_features(
    dataframes, 
    sequences,
    lags=range(1, forecast_horizon+ 1)
)

features = all_features.copy()


In [ ]:
#Step 2: Feature Selection
#to be implemented
print('Feature Selection: To be implemented')

In [ ]:
#step 3: Standardize the data and apply PCA
#to be implemented
'''
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Assuming 'all_features' is your DataFrame with the variables you want to analyze
X = all_features.bfill() 
# Step 1: Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#display and plot scaled data
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

filtered_features = all_features

X_scaled 
'''
print('Standardize the data and apply PCA: To be implemented')


In [ ]:
#Step 4: Data Preparation

# Prepare the target variable

#-------------------------------------------------------------
series_returns = pd.DataFrame(data[ticker]['full']['Close'].pct_change(forecast_horizon)).bfill()
series_returns_ts = TimeSeries.from_dataframe(series_returns)

split_point = int(len(series_returns_ts) * train_percentage)
train_series_returns_ts = series_returns_ts[:split_point]
test_series_returns_ts  = series_returns_ts[split_point:]
#-------------------------------------------------------------


# Prepare the features

#-------------------------------------------------------------
features = all_features.bfill()
features_ts = TimeSeries.from_dataframe(features)

scaler_features_ts = Scaler()
features_ts_normalized = scaler_features_ts.fit_transform(features_ts)

split_point = int(len(features_ts_normalized) * train_percentage)
train_features_ts_normalized = features_ts_normalized[:split_point]
test_features_ts_normalized  = features_ts_normalized[split_point:]
#-------------------------------------------------------------

# Debugging code
print("Train series returns shape:", train_series_returns_ts.pd_dataframe().shape)
print("Test series returns shape:", test_series_returns_ts.pd_dataframe().shape)
print("Train features shape:", train_features_ts_normalized.pd_dataframe().shape)
print("Test features shape:", test_features_ts_normalized.pd_dataframe().shape)
print(' ')

#check nan values
print('Train series returns nan values:', train_series_returns_ts.pd_dataframe().isnull().sum().sum())
print('Test series returns nan values:', test_series_returns_ts.pd_dataframe().isnull().sum().sum())
print('Train features nan values:', train_features_ts_normalized.pd_dataframe().isnull().sum().sum())
print('Test features nan values:', test_features_ts_normalized.pd_dataframe().isnull().sum().sum())

# Ensure lengths match
assert len(train_series_returns_ts) == len(train_features_ts_normalized), "Train series and features length mismatch."
assert len(test_series_returns_ts) == len(test_features_ts_normalized), "Test series and features length mismatch."

In [26]:
#Model & Prediction: Linear Regression
from darts.models import LinearRegressionModel
from darts.models import LinearRegressionModel
from darts.utils import statistics
from darts.metrics import mape, mae
from darts.utils.likelihood_models import GaussianLikelihood

def train_fit_predict_linear_regression_model(target, past_cov, forecast_horizon):
    model = LinearRegressionModel(
        lags=12,
        lags_past_covariates=12,
        output_chunk_length=forecast_horizon,
        n_jobs=-1
    )
    
    model.fit(
        target,
        past_covariates=past_cov,
    )
    
    pred = model.predict(n=forecast_horizon)
    return model, pred


#prediction the full set
model, pred_full = train_fit_predict_linear_regression_model(series_returns_ts, features_ts_normalized, forecast_horizon)

#backtest the model (not retraining)
backtest_predictions = model.historical_forecasts(
    series_returns_ts,
    past_covariates=features_ts_normalized,
    forecast_horizon=forecast_horizon,
    retrain=False,
    enable_optimization=True
)


#plot the backtest results
plot_backtest_results_plotly(series_returns_ts, backtest_predictions)

#plot the forecast for the full set
plot_forecast(series_returns_ts, pred_full)


In [30]:
#Model & Prediction:: NBeats or Nhits (with probabilistic forecasts)
from darts.models import NHiTSModel

def train_fit_predict_nhits_model(target, past_cov, forecast_horizon):
    """
    Train, fit, and predict using the NHiTS model.
    
    Args:
        target (TimeSeries): The target TimeSeries.
        past_cov (TimeSeries): The past covariates TimeSeries.
        forecast_horizon (int): The number of steps to forecast.
    
    Returns:
        model (NHiTSModel): The trained NHiTS model.
        pred (TimeSeries): The forecasted TimeSeries.
    """
    

    model = NHiTSModel(
        input_chunk_length=12,
        output_chunk_length=forecast_horizon,
        num_blocks=4,
        num_layers=4,
        n_epochs=100,
        nr_epochs_val_period=10,
        batch_size=32,
        model_name="NHiTS_Model",
    )
    
    model.fit(
        series=target,
        past_covariates=past_cov,
    )
    
    pred = model.predict(n=forecast_horizon)
    return model, pred



#prediction the full set
model, pred_full = train_fit_predict_nhits_model(series_returns_ts, features_ts_normalized, forecast_horizon)

#backtest the model (not retraining)
backtest_predictions = model.historical_forecasts(
    series_returns_ts,
    past_covariates=features_ts_normalized,
    forecast_horizon=forecast_horizon,
    retrain=False,
    enable_optimization=True
)


#plot the backtest results
plot_backtest_results_plotly(series_returns_ts, backtest_predictions)

#plot the forecast for the full set
plot_forecast(series_returns_ts, pred_full)

In [315]:
#Model: Torch Model (LSTM)

In [316]:
#Model: Transformer Model